In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import sampler, TensorDataset, Dataset
import test_model

import math
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from pandas import Series
import random
import scipy
import cv2
from scipy.stats.stats import pearsonr
from sklearn.metrics import mean_absolute_error, r2_score
import time

ImportError: No module named 'test_model'

In [ ]:
df = pd.read_excel('Test_Info.xlsx')
for index, row in df.iterrows():
    df.iloc[index, 0] = df.iloc[index, 0].split("'")[1]

In [ ]:
# from sklearn.utils import shuffle
# df_sub = df[df['AGE'].between(0, 35, inclusive = (True, False))]
# df_sub = shuffle(df_sub)
# df_sub = df_sub.iloc[:7, :]
# ttt = list(df_sub.ID.values)

In [ ]:
# df = pd.DataFrame(row for index, row in df.iterrows() if row.ID not in ttt)

In [ ]:
# df = df.reset_index()

In [ ]:
dirn_MD = "/home/lwk/MRI/testing/dti_nlmeans/MD_nl4/"
dirn_FA = "/home/lwk/MRI/testing/dti_nlmeans/FA_nl4/"
# dirn_MD = "/data/put_data/lwk/MRI/nlmeans_32/MD_DTI/"
# dirn_FA = "/data/put_data/lwk/MRI/nlmeans_32/FA_DTI/"

df2 = pd.read_csv('TrainingDataDemographics_clean.csv')
plt.hist(df2.AGE)
plt.savefig("dis_all.png" , dpi = 200)
plt.show()
plt.close()

df3 = pd.read_csv('Test_115.csv')
plt.hist(df3.AGE)
plt.savefig("dis_test_115.png" , dpi = 200)
plt.show()
plt.close()

plt.hist(df.AGE)
plt.savefig("dis_test_50.png" , dpi = 200)
plt.show()
plt.close()

##### minv = np.min(df.AGE.values)
maxv = np.max(df.AGE.values)
bins = 5
plt.hist(df.AGE,bins=bins, normed=True, cumulative=True, label='CDF DATA', histtype='step', alpha=0.55, color='red') 
plt.hist(df2.AGE,bins=bins, normed=True, cumulative=True, label='CDF DATA', histtype='step', alpha=0.55, color='green') 
plt.hist(df3.AGE,bins=bins, normed=True, cumulative=True, label='CDF DATA', histtype='step', alpha=0.55, color='blue') 
plt.axis([10, maxv, 0, 1])
plt.xlabel('Age')
plt.ylabel('Cumulative probability')
plt.legend(['NTU test data (counts=50)', 'All data (counts=558)', 'Test data (counts=115)'], loc='upper left')
plt.savefig("CDF.png" , dpi = 300)
plt.show()
plt.close()

df['data'] = 'NTU test data (counts=50)'
df2['data'] = 'All data (counts=558)'
df3['data'] = 'Test data (counts=115)'
data = pd.concat((df, df2.iloc[:, 1:], df3.iloc[:, 1:]), 0)
data.boxplot(column = 'AGE', by = 'data',figsize=(20,10), grid=False)
plt.savefig('box.png', dpi=300)

minv

In [ ]:
test_id = df.ID
print(len(test_id))

## model loading

In [ ]:
torch.cuda.set_device(0)
# specify dtype
use_cuda = torch.cuda.is_available()
print(use_cuda)
if use_cuda:
    dtype = torch.cuda.FloatTensor
else:
    dtype = torch.FloatTensor
print(torch.cuda.device_count())

In [ ]:
# 1. model_for_prediction/1208/bottle_FA_MD_C16_A3_aug_nlmean_RMSE MAE=4.636 R2=88.376
# 2. model_for_prediction/1215/bottle_FA_MD_C16_A3_aug_nlmean32_RMSE_adam
dir_m = 'model_for_prediction/1219/bottle_FA_MD_C16_A3_aug_nlmeans_RMSE_adam/'

In [ ]:
time_str = time.strftime("%m%d")
result_dirn = 'result/' + time_str + '/' + dir_m.split('/')[-2] + '/' 
if not os.path.exists(result_dirn):
    os.makedirs(result_dirn)
print('result will be saved to ', result_dirn)

In [ ]:
# model = test_model.CNN_1(base=16).type(dtype)
model = test_model.CNN_2(base=32).type(dtype)
net = torch.nn.DataParallel(model)

In [ ]:
net.load_state_dict(torch.load(dir_m + 'model.pkl'))

## Data loader

In [ ]:
def change_format(img):
    img = np.expand_dims(img, axis = 3)
    img = np.swapaxes(np.swapaxes(img, 1, 2), 0, 3)
    return img

In [ ]:
class MDFA_Dataset(Dataset):
    def __init__(self, id_list, dirn_MD, dirn_FA):
        self.id_list = id_list
        self.dirn_MD = dirn_MD
        self.dirn_FA = dirn_FA
    def __getitem__(self, index):
        X_MD = np.load(self.dirn_MD + self.id_list[index] + '.npy')
        X_FA = np.load(self.dirn_FA + self.id_list[index] + '.npy')
        X_MD = change_format(X_MD)
        X_FA = change_format(X_FA)
        label = df[df['ID'] == self.id_list[index]]['AGE'].values[0]
        return X_MD, X_FA, label

    def __len__(self):
        return len(self.id_list)

In [ ]:
test_MDFA_dataset = MDFA_Dataset(test_id, dirn_MD, dirn_FA)
testloader = torch.utils.data.DataLoader(test_MDFA_dataset, batch_size=len(test_id), shuffle=False, num_workers=2)

## Prediction

In [ ]:
def plot_making(true, pred, types):
    cor = pearsonr(true, pred)[0]
    mae = mean_absolute_error(true, pred)
    r2 = r2_score(true, pred) 
    plt.figure(0)
    plt.plot(true, true,'k-')
    plt.scatter(true, pred, alpha = .15, s = 20)
    plt.xlabel('True_Y')
    plt.ylabel('Pred_Y')
    plt.title("MAE = %3f; Cor = %3f; R2 = %3f; #samples = %d" % (mae, cor, r2, len(true)))
    plt.savefig(result_dirn + types + "_plot_scatter.png" , dpi = 200)
    plt.show()
    plt.close()

In [ ]:
criterion = nn.L1Loss()
def loss_generator(testloader, model, types):
    test_loss = []
    true = []
    pred = []
    for i, tdata in enumerate(testloader, 0):
        tinputs1,tinputs2, tlabels = tdata
        tinputs1,tinputs2, tlabels = Variable(tinputs1, volatile=True).type(dtype), Variable(tinputs2, volatile=True).type(dtype), Variable(tlabels, volatile=True).type(dtype)
        toutput = net(tinputs1,tinputs2)
        tloss = criterion(toutput, tlabels)
        test_loss.append(tloss.data[0])
        true.extend(tlabels.data.cpu().numpy())
        pred.extend(toutput.data.cpu().numpy())
    true = np.asarray(true)
    pred = np.asarray(pred)
    pred = np.squeeze(pred, axis = 1)
    d = {'true': true, 'pred': pred}
    df = pd.DataFrame(data=d)
    df.to_csv('out_115.csv')
    plot_making(true, pred, types = types)
    print('L1 Loss on test images: %r' % (np.average(test_loss)))

loss_generator(testloader, net.eval(), types = 'test')

In [ ]:
df